# Creating Our Dataset and Applying Our Transformer

In this notebook, we:
- convert the Friends dataset (https://github.com/emorynlp/character-mining) into a Corpus with ConvoKit
- apply our Transformer, Genderromantic, to the data
- analyze the resulting measures

In [1]:
!python3 -m pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/69/1b/b853c7a9d4f6a6d00749e94eb6f3a041e342a885b87340b79c1ef73e3a78/certifi-2019.6.16-py2.py3-none-any.whl
  Found existing installation: certifi 2019.9.11
    Uninstalling certifi-2019.9.11:
      Successfully uninstalled certifi-2019.9.11


In [2]:
import convokit

In [3]:
convokit

<module 'convokit' from '/Users/emilytseng/Cornell-Conversational-Analysis-Toolkit/convokit/__init__.py'>

In [4]:
import requests
import json
from tqdm import tqdm
from convokit import Corpus, User, Utterance

## The Friends Dataset

The original dataset (https://github.com/emorynlp/character-mining) contains a set of 10 JSON files, each of which represents a complete transcript of 1 season of <i>Friends</i>. Since the data are available in JSON format from this GitHub repo, we download the raw data directly using the `requests` module. You will not need to download raw data files to use this script.

## Generating user information
Since our dataset doesn't have any existing user information, we extract speaker information from the conversation. For each user, we collect the episode in which he/she first appears and guess his/her gender based on the name using the gender_guesser module.

Users are indexed by their name, which is a `<str>`. For each user, we create an object with:

- <b>first_appearance:</b> the episode in which he or she first appeared
- <b>gender:</b> the character's gender, as defined by the `gender_guesser` module's guess of his/her name

In [6]:
! pip3 install gender_guesser
import gender_guesser.detector as gender
d = gender.Detector()

In [11]:
users = {}
for i in tqdm(range(1,11)):
  season_number = '0'+str(i) if i < 10 else '10'
  json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_'+str(season_number)+'.json'
  r = requests.get(json_file)
  
  season = json.loads(r.text)
  episodes = season['episodes']
  for j in range(len(episodes)):
    episode = episodes[j]
    scenes = episode['scenes']
    for k in range(len(scenes)):
      scene = scenes[k]
      utterances = scene['utterances']
      for l in range(len(utterances)):
        utterance = utterances[l]
        speaker_list = utterance['speakers']
        for speaker in speaker_list:
          if speaker not in users:
            users[speaker] = {'first_appearance': episode['episode_id'], 'gender': d.get_gender(speaker.split()[0])}

100%|██████████| 10/10 [00:07<00:00,  1.37it/s]


Sanity-checking the user data, we should see the correct genders assigned to the 6 friends:

In [12]:
print("number of users in the data = {}/700".format(len(users)))
print("Monica Geller object: ", users["Monica Geller"])
print("Joey Tribbiani object: ", users["Joey Tribbiani"])
print("Chandler Bing object: ", users["Chandler Bing"])
print("Phoebe Buffay object: ", users["Phoebe Buffay"])
print("Ross Geller object: ", users["Ross Geller"])
print("Rachel Green object: ", users["Rachel Green"])

number of users in the data = 700/700
Monica Geller object:  {'first_appearance': 's01_e01', 'gender': 'female'}
Joey Tribbiani object:  {'first_appearance': 's01_e01', 'gender': 'male'}
Chandler Bing object:  {'first_appearance': 's01_e01', 'gender': 'mostly_male'}
Phoebe Buffay object:  {'first_appearance': 's01_e01', 'gender': 'female'}
Ross Geller object:  {'first_appearance': 's01_e01', 'gender': 'male'}
Rachel Green object:  {'first_appearance': 's01_e01', 'gender': 'female'}


We then create a User object for each unique character in the dataset.

In [13]:
corpus_users = {k: User(name=k, meta=v) for k,v in users.items()}

In [14]:
print(corpus_users['Monica Geller'].name)
print(corpus_users['Monica Geller'].meta)

Monica Geller
{'first_appearance': 's01_e01', 'gender': 'female'}


## Generating Utterances

We then loop through the data to generate a list of all utterances in the series. To align with the Utterance schema ConvoKit expects, we construct for each utterance:

- **id:** index of the utterance

- **user:** the user who authored the utterance; the speaker in our case

- **root:** id of the conversation root of the utterance; the first utterance in the scene, in our case

- **reply_to:** id of the utterance to which this utterance replies to; None if the utterance is not a reply.

- **timestamp:** time of the utterance (None for us -- the dataset does not contain this information)

- **text:** textual content of the utterance

We also pull in the following metadata including:
- **tokens** a tokenized representation of the text (handy for sentence separation)
-**character_entities** available for some but not all utterances; `None` if unavailable. These are intended to identify who the user is speaking to and/or about.
-**emotion** emotion labels for each token. Available for some but not all utterances; `None` if unavailable. 
-**caption**  available for some but not all utterances; `None` if unavailable. This contains the begin time, end time, and text sans punctuation. Only available for seasons 6-9.
-**transcript_with_note**  a version of the text with an action note (e.g. "(to Ross) Hand me the coffee" vs. "Hand me the coffee"). Available for some but not all utterances; `None` if unavailable.
-**token_with_note** a tokenized representation of the above.

In [15]:
all_utterances = {}



for i in tqdm(range(1,11)):
  season_number = '0'+str(i) if i < 10 else '10'
  json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_'+str(season_number)+'.json'
  r = requests.get(json_file)
  
  season = json.loads(r.text)
  episodes = season['episodes']
  for j in range(len(episodes)):
    episode = episodes[j]
    scenes = episode['scenes']
    for k in range(len(scenes)):
      scene = scenes[k]
      utterances = scene['utterances']
      
      root = utterances[0] #set the root as the first utterance in the scene for now
      
      prev_utt = None

      for l in range(len(utterances)):
        utterance = utterances[l]
        
        speaker = utterance['speakers']
        
        if len(speaker) == 0:
          prev_utt = None
          continue
        
        # Add meta       
        meta = {
            'tokens': utterance.get('tokens'),
            'character_entities': utterance.get('character_entities'),
            'emotion': utterance.get('emotion'),
            'caption': utterance.get('caption'),
            'transcript_with_note': utterance.get('transcript_with_note'),
            'tokens_with_note': utterance.get('tokens_with_note')
        }
        
        # Create the Utterance, including meta
        all_utterances[utterance['utterance_id']] = Utterance(
            id=utterance['utterance_id'],
            user=corpus_users[speaker[0]],
            root=root['utterance_id'],
            reply_to=prev_utt,
            timestamp=None,
            text=utterance['transcript'],
            meta=meta
        )
        
        # Get the prev_utt for the next iteration
        prev_utt = utterance['utterance_id']


100%|██████████| 10/10 [00:04<00:00,  2.11it/s]


In [16]:
print("This corpus has {}/61309 utterances".format(len(all_utterances)))

This corpus has 61338/61309 utterances


In [17]:
all_utterances['s01_e18_c05_u021']

Utterance({'id': 's01_e18_c05_u021', 'user': User([('name', 'Ross Geller')]), 'root': 's01_e18_c05_u001', 'reply_to': 's01_e18_c05_u020', 'timestamp': None, 'text': 'Alright.', 'meta': {'tokens': [['Alright', '.']], 'character_entities': [[]], 'emotion': ['Neutral', ['Neutral', 'Neutral', 'Neutral', 'Neutral']], 'caption': None, 'transcript_with_note': None, 'tokens_with_note': None}})

## Creating the corpus from a list of utterances

We now create the corpus from our dict of utterances. Note, we are are allowing convokit to create conversations IDs automatically after loading the utterances list.

In [18]:
utterance_list = [utt for k, utt in all_utterances.items()]

In [19]:
friends_corpus = Corpus(utterances=utterance_list, version=1)

Sanity checks for the number of conversations in the dataset and the first 5 conversations:

In [20]:
print("number of conversations in the dataset={}".format(len(friends_corpus.get_conversation_ids())))

number of conversations in the dataset=3099


In [21]:
convo_ids = friends_corpus.get_conversation_ids()
for i, convo_idx in enumerate(convo_ids[0:5]):
    print("sample conversation {}:".format(i))
    print(friends_corpus.get_conversation(convo_idx).get_utterance_ids())

sample conversation 0:
['s01_e01_c01_u001', 's01_e01_c01_u002', 's01_e01_c01_u003', 's01_e01_c01_u004', 's01_e01_c01_u006', 's01_e01_c01_u007', 's01_e01_c01_u008', 's01_e01_c01_u010', 's01_e01_c01_u011', 's01_e01_c01_u012', 's01_e01_c01_u013', 's01_e01_c01_u014', 's01_e01_c01_u015', 's01_e01_c01_u016', 's01_e01_c01_u017', 's01_e01_c01_u018', 's01_e01_c01_u019', 's01_e01_c01_u021', 's01_e01_c01_u022', 's01_e01_c01_u023', 's01_e01_c01_u024', 's01_e01_c01_u025', 's01_e01_c01_u026', 's01_e01_c01_u027', 's01_e01_c01_u028', 's01_e01_c01_u029', 's01_e01_c01_u030', 's01_e01_c01_u031', 's01_e01_c01_u032', 's01_e01_c01_u033', 's01_e01_c01_u034', 's01_e01_c01_u035', 's01_e01_c01_u036', 's01_e01_c01_u037', 's01_e01_c01_u038', 's01_e01_c01_u039', 's01_e01_c01_u040', 's01_e01_c01_u041', 's01_e01_c01_u042', 's01_e01_c01_u044', 's01_e01_c01_u045', 's01_e01_c01_u047', 's01_e01_c01_u048', 's01_e01_c01_u049', 's01_e01_c01_u050', 's01_e01_c01_u051', 's01_e01_c01_u052', 's01_e01_c01_u053', 's01_e01_c01_u05

Summary stats for the corpus:

In [22]:
friends_corpus.print_summary_stats()

Number of Users: 699
Number of Utterances: 61338
Number of Conversations: 3099


## Adding corpus-level metadata

We add the name of the corpus.

In [23]:
friends_corpus.meta['name'] = 'Friends Dataset'

# Use locally defined Transformer

In [56]:
import convokit
from convokit import Genderromantic
Genderromantic

convokit.genderromantic.genderromantic.Genderromantic

In [57]:
grr = Genderromantic()


In [58]:
grr.fit_transform(friends_corpus)

FileNotFoundError: [Errno 2] No such file or directory: 'RomanticWords.txt'

In [41]:
import inspect

In [45]:
inspect.getmembers(Genderromantic)

[('__abstractmethods__', frozenset()),
 ('__class__', abc.ABCMeta),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  mappingproxy({'__module__': 'convokit.genderromantic.genderromantic',
                '__init__': <function convokit.genderromantic.genderromantic.Genderromantic.__init__(self)>,
                'genderDictionary': <staticmethod at 0x11792b750>,
                'transform': <function convokit.genderromantic.genderromantic.Genderromantic.transform(self, corpus: convokit.model.corpus.Corpus)>,
                '__doc__': None,
                '__abstractmethods__': frozenset(),
                '_abc_impl': <_abc_data at 0x117911db0>})),
 ('__dir__', <method '__dir__' of 'object' objects>),
 ('__doc__', None),
 ('__eq__', <slot wrapper '__eq__' of 'object' objects>),
 ('__format__', <method '__format__' of 'object' objects>),
 ('__ge__', <slot wrapper '__ge__' of 'object' objects>),
 ('__getattribute__', <slot wrapper '__getattribute__' of 